In [228]:
!pip install memory_profiler
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [229]:
import pandas as pd, random, os, csv, time
from collections import defaultdict


In [230]:
# === PARAMS ===
CSV_PATH = "/content/sample_data/jadwal_kuliah_random_fix9_halfhour.csv"
N_STUDENTS = 300

REQUIRED = {"EF234501","EF234502","EF234503","EF234504","EF234505"}
ELECTIVES = {"EF234509","EF234518","EF234713","EK234601","EF234618",
             "EF234606","ER234504","EF234614","EF234704","EF234708"}

import random, os, csv, time
from collections import defaultdict

RANDOM_SEED = int(time.time()) % 100000
random.seed(RANDOM_SEED)
print("🎲 Random seed:", RANDOM_SEED)

def to_minutes(hhmm: str) -> int:
    h, m = map(int, hhmm.split(":"))
    return h*60 + m

def conflict(secA, secB) -> bool:
    # day equal AND time overlaps
    return (secA["day"] == secB["day"]) and not (secA["end"] <= secB["start"] or secB["end"] <= secA["start"])

def load_schedule(csv_path):
    sections = {}
    with open(csv_path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for r in reader:
            code = r["Kode Kelas"].split()[0]
            cap = 60 if code in REQUIRED else 30
            sections[r["Kode Kelas"]] = {
                "code": code,
                "name": r["Nama Mata Kuliah"],
                "day": r["Hari"],
                "start": to_minutes(r["Mulai"]),
                "end": to_minutes(r["Selesai"]),
                "dur": int(r["Durasi (menit)"]),
                "cap": cap,
            }
    return sections

def group_by_course(sections):
    g = defaultdict(list)
    for sid, s in sections.items():
        g[s["code"]].append(sid)
    return g


🎲 Random seed: 89613


In [231]:
sections = load_schedule(CSV_PATH)
course_to_secs = group_by_course(sections)
capacity = {sid: s["cap"] for sid, s in sections.items()}

# sanity checks
total_req_seats = sum(capacity[sid] for sid, s in sections.items() if s["code"] in REQUIRED)
total_elec_seats = sum(capacity[sid] for sid, s in sections.items() if s["code"] in ELECTIVES)
print("Seats Wajib:", total_req_seats, "(harus 1500)")
print("Seats Pilihan:", total_elec_seats, "(harus 600)")
print("Jumlah section:", len(sections))


Seats Wajib: 1500 (harus 1500)
Seats Pilihan: 600 (harus 600)
Jumlah section: 45


In [232]:
def assign_student_backtracking(capacity):
    # 5 wajib + 2 pilihan (random distinct)
    courses = list(REQUIRED) + random.sample(list(ELECTIVES), 2)

    assigned = {}          # course_code -> section_id
    chosen_secs = []       # list of chosen section_id (for conflict check)

    # urutkan berdasarkan banyak nya ketersediaan kelas (mulai dari paling sedikit)
    sorted_courses = sorted(courses, key=lambda c: sum(1 for sid in course_to_secs[c] if capacity[sid] > 0))

    def backtrack(i: int) -> bool:
        if i == len(sorted_courses):
            return True
        course = sorted_courses[i]
        # urutkan berdasarkan kapasitas paling banyak
        options = [sid for sid in course_to_secs[course] if capacity[sid] > 0]
        options.sort(key=lambda sid: capacity[sid], reverse=True)
        for sid in options:
            s = sections[sid]
            if any(conflict(s, sections[cid]) for cid in chosen_secs):
                continue
            # pilih
            assigned[course] = sid
            chosen_secs.append(sid)
            capacity[sid] -= 1

            if backtrack(i+1):
                return True

            # batal
            capacity[sid] += 1
            chosen_secs.pop()
            assigned.pop(course, None)
        return False

    ok = backtrack(0)
    return assigned if ok else None


In [233]:
%%memit

out_dir = "/content/output_backtracking"
os.makedirs(out_dir, exist_ok=True)

start = time.time()
success = 0

for i in range(1, N_STUDENTS+1):
    sid = f"S{i:03d}"
    alloc = None
    # beberapa percobaan supaya tidak buntu (ubah angka kalau perlu)
    for _ in range(500):
        alloc = assign_student_backtracking(capacity)
        if alloc:
            break
    if not alloc:
        print(f"[!] Gagal alokasi {sid} (coba tambah percobaan/ubah seed)")
        continue

    # tulis CSV per mahasiswa
    # sort tampilan per hari & jam mulai
    day_order = {"Senin":1,"Selasa":2,"Rabu":3,"Kamis":4,"Jumat":5}
    rows = []
    for course_code, sec_id in alloc.items():
        info = sections[sec_id]
        jenis = "Wajib" if course_code in REQUIRED else "Pilihan"
        rows.append([info["day"],
                     f"{info['start']//60:02d}:{info['start']%60:02d}",
                     f"{info['end']//60:02d}:{info['end']%60:02d}",
                     sec_id, info["name"], jenis])
    rows.sort(key=lambda r: (day_order[r[0]], r[1]))

    with open(os.path.join(out_dir, f"{sid}.csv"), "w", encoding="utf-8") as f:
        f.write("Hari,Mulai,Selesai,Kode Kelas,Nama Mata Kuliah,Jenis\n")
        for r in rows:
            f.write(",".join(r) + "\n")
    success += 1
    if i % 25 == 0:
        print(f"Progress: {i}/{N_STUDENTS} (sukses: {success})")

elapsed = time.time() - start
print(f"Done. Sukses: {success}/{N_STUDENTS}. Waktu: {elapsed:.2f}s")


Progress: 25/300 (sukses: 25)
Progress: 50/300 (sukses: 50)
Progress: 75/300 (sukses: 75)
Progress: 100/300 (sukses: 100)
Progress: 125/300 (sukses: 125)
Progress: 150/300 (sukses: 150)
Progress: 175/300 (sukses: 175)
Progress: 200/300 (sukses: 200)
Progress: 225/300 (sukses: 225)
Progress: 250/300 (sukses: 250)
Progress: 275/300 (sukses: 275)
[!] Gagal alokasi S298 (coba tambah percobaan/ubah seed)
[!] Gagal alokasi S299 (coba tambah percobaan/ubah seed)
[!] Gagal alokasi S300 (coba tambah percobaan/ubah seed)
Done. Sukses: 297/300. Waktu: 0.09s
peak memory: 310.39 MiB, increment: 0.01 MiB


In [234]:
# out_path="/content/output_backtracking_full_zip"
# os.makedirs(out_path,exist_ok=True)

# for sid,alloc in assignments.items():
#     day_order={"Senin":1,"Selasa":2,"Rabu":3,"Kamis":4,"Jumat":5}
#     rows=[]
#     for c,sec in alloc.items():
#         info=sections[sec]; jenis="Wajib" if c in REQUIRED else "Pilihan"
#         rows.append([info["day"],
#                      f"{info['start']//60:02d}:{info['start']%60:02d}",
#                      f"{info['end']//60:02d}:{info['end']%60:02d}",
#                      sec,info["name"],jenis])
#     rows.sort(key=lambda r:(day_order.get(r[0],9),r[1]))
#     with open(os.path.join(out_path,f"{sid}.csv"),"w",encoding="utf-8") as f:
#         f.write("Hari,Mulai,Selesai,Kode Kelas,Nama Mata Kuliah,Jenis\n")
#         for r in rows: f.write(",".join(r)+"\n")

# zip_path="/content/jadwal_backtracking_full_300.zip"
# with zipfile.ZipFile(zip_path,"w",zipfile.ZIP_DEFLATED) as zf:
#     for fn in sorted(os.listdir(out_path)):
#         zf.write(os.path.join(out_path,fn),arcname=fn)
# print("📦 ZIP ready:", zip_path)
